In [33]:
import scipy
import bagpy
import numpy as np
from bagpy import bagreader
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pylab as pylab
from matplotlib import rc
import statistics
import regex as re
rc('text', usetex=False) 

### Plotting methods for the Fundamentals of Robotics Course

In [69]:
def get_topics(b):
    csv_files = []
    for topic in b.topics:
        #print(topic)
        data = b.message_by_topic(topic)
        csv_files.append(data)
    return csv_files

bag = "./recorded_data.bag"
b = bagreader(bag)
csv_files = get_topics(b)
joint_states = pd.read_csv(csv_files[0])
ho_cook_joint_states = pd.read_csv(csv_files[1])
p2p_joint_states = pd.read_csv(csv_files[2])
joint_trajectory = pd.read_csv(csv_files[-1])
print(csv_files)

def parse_timeseries_string(string_data):
    # Define regular expressions for extracting positions and velocities
    positions_pattern = re.compile(r'positions: \[([^\]]+)\]')
    velocities_pattern = re.compile(r'velocities: \[([^\]]+)\]')
    time_pattern = re.compile(r'time_from_start: \n\s*secs: (\d+)\n\s*nsecs: (\d+)')

    # Extract data using regular expressions
    positions_match = positions_pattern.findall(string_data)
    velocities_match = velocities_pattern.findall(string_data)
    time_match = time_pattern.findall(string_data)

    # Initialize lists for positions and velocities
    positions_list = []
    velocities_list = []
    times_list = []

    # Extract positions
    for positions_str in positions_match:
        positions_list.append([float(pos) for pos in positions_str.split(',')])

    # Extract velocities (assuming an empty list if not found)
    for velocities_str in velocities_match:
        velocities_list.append([float(vel) for vel in velocities_str.split(',')])
        
    # Extract time_from_start
    for secs, nsecs in time_match:
        time_in_seconds = int(secs) + int(nsecs) / 1e9
        times_list.append(time_in_seconds)

    return positions_list, velocities_list, times_list

def parse_timeseries(timeseries):
    positions_list = []
    times_list = []

    for entry in timeseries:
        positions = entry.get('positions', [])
        time_from_start = entry.get('time_from_start', {})
        secs = time_from_start.get('secs', 0)
        nsecs = time_from_start.get('nsecs', 0)
        time_in_seconds = secs + nsecs / 1e9  # Convert nanoseconds to seconds

        positions_list.append(positions)
        times_list.append(time_in_seconds)

    return positions_list, times_list

def parse_joint_states(joint_states): 
    
    n_joints = len(joint_states["name"][0].split(","))
    q = {}
    q["t"] = joint_states["Time"]
    print(n_joints)
    for i in range(0, n_joints): 
        q["p{}".format(i)] = joint_states["position_{}".format(i)]
        q["v{}".format(i)] = joint_states["velocity_{}".format(i)]
    return q

import matplotlib.pyplot as plt

def plot_joint_positions_velocities(positions, velocities, times):
    # Plot Joint Positions
    plt.figure(figsize=(12, 6))
    for i in range(len(positions[0])):
        plt.plot(times, [pos[i] for pos in positions], label=f'Joint {i+1} Position')

    plt.title('Joint Positions Over Time')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Joint Positions')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot Joint Velocities
    plt.figure(figsize=(12, 6))
    for i in range(len(velocities[0])):
        plt.plot(times, [vel[i] for vel in velocities], label=f'Joint {i+1} Velocity')

    plt.title('Joint Velocities Over Time')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Joint Velocities')
    plt.legend()
    plt.grid(True)
    plt.show()
    
# Commanded points for HoCook 
def parse_trajectory(trajectory): 
    p, v, t = parse_timeseries_string(trajectory) 
    return p, v, t

[INFO]  Data folder ./recorded_data already exists. Not creating.
['./recorded_data/hocook-joint_states.csv', './recorded_data/joint_states.csv', './recorded_data/p2p-joint_states.csv', './recorded_data/position_joint_trajectory_controller-command.csv']


In [49]:
q = parse_joint_states(joint_states)

7


In [46]:

p_, v_, t = parse_timeseries_string(joint_trajectory['points'][3])
q0 = [p[0] for p in p_]; v0 = [v[0] for v in v_]
q1 = [p[1] for p in p_]; v1 = [v[1] for v in v_]
q2 = [p[2] for p in p_]; v2 = [v[2] for v in v_]
q3 = [p[3] for p in p_]; v3 = [v[3] for v in v_]
q4 = [p[4] for p in p_]; v4 = [v[4] for v in v_]
q5 = [p[5] for p in p_]; v5 = [v[5] for v in v_]
q6 = [p[6] for p in p_]; v6 = [v[6] for v in v_]
t.insert(0, 0)

In [74]:
parsed_trajectories = {}
for i, t in enumerate(joint_trajectory['points']): 
    p_, v_, t = parse_trajectory(joint_trajectory['points'][i])
    parsed_trajectories["{}".format(f"{i}")] = {"p": p_, "v": v_, "t": t}
    

In [80]:
for key in parsed_trajectories.keys(): 
    print(parsed_trajectories[f'{key}']['t'])

[]
[]
[]
[]
[]
[]
[]
[0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.56, 0.58, 0.6, 0.62, 0.64, 0.66, 0.68, 0.7, 0.72, 0.74, 0.76, 0.78, 0.8, 0.82, 0.84, 0.86, 0.88, 0.9, 0.92, 0.94, 0.96, 0.98, 1.1, 1.12, 1.1400000000000001, 1.159999999, 1.179999999, 1.199999999, 1.2199999990000001, 1.24, 1.26, 1.28, 1.3, 1.32, 1.34, 1.3599999999999999, 1.38, 1.4, 1.419999999, 1.4399999989999999, 1.4599999989999999, 1.48, 1.5, 1.52, 1.54, 1.56, 1.58, 1.6, 1.62, 1.6400000000000001, 1.6600000000000001, 1.679999999, 1.7, 1.72, 1.74, 1.76, 1.78, 1.8, 1.8199999999999998, 1.8399999999999999, 1.8599999999999999, 1.88, 1.9, 1.9199999989999998, 1.94, 1.96, 1.98, 2.1, 2.12, 2.14, 2.16, 2.18, 2.2, 2.22, 2.24, 2.26, 2.2800000000000002, 2.3, 2.319999999, 2.3399999989999998, 2.3599999990000002, 2.379999999, 2.399999999, 2.419999999, 2.439999999, 2.459999999, 2.48, 2.5, 2.52, 2.54, 2.56, 2.58, 2.6, 2.62, 2.64, 2.66, 2.68, 2.7, 2.